In [1]:
import numpy as np
import pandas as pd
import scipy.io
from scipy import special
from scipy import optimize
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.express as px
from datetime import date
from glob import glob
from os.path import join, exists
import random
from random import random as rand

import clmnlab

/home/sungbeenpark/anaconda3/envs/GA/lib/python3.9/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
GA = clmnlab.GA()

OS : linux
Google Drive is detected!
Git directory is detected!


In [3]:
%config InlineBackend.figure_format = 'retina'
py.offline.init_notebook_mode(connected = True)

In [4]:
np.set_printoptions(precision=5, suppress=True)

---

In [8]:
datum = scipy.io.loadmat(fname)
# datum

In [9]:
## frame per trial
fpt = int(datum['nSampleTrial'][0][0])
## second per trial
spt = int(fpt/60.)
## a number of trials per run
# tpr = datum['nTrialperRun'][0][0]
tpr = 97  # 12 trial/block * 8 block/run + 1 = 97 trial/run
## target ID
tmp = datum['targetID'][0]
targetID = tmp[tmp!=0][:tpr]
# targetID = tmp
nrun = 3
# ntrial = 12
# nblock = 8

In [11]:
tmp = datum['targetID'][0]
targetID = tmp[tmp!=0][:tpr*nrun]
targetID.shape

(291,)

In [12]:
bx = datum['boxSize'][0][0]
pos = bx*GA.convert_ID(targetID)
pos.shape

(2, 291)

60 Hz * {5 s/trial * (1 trial + 12 trial/block * 8 block)}/Run * 3 Run = 87300

In [13]:
datum['allXY'].shape

(2, 87300)

In [14]:
allXY = datum['allXY']
allXY.shape

(2, 87300)

In [15]:
xFrame, yFrame = np.array([datum['xFrame'].squeeze(), datum['yFrame'].squeeze()]).astype(int)
print(xFrame, yFrame)

345 345


Is it hit?

In [16]:
## target position
tmp = np.zeros(allXY.shape)
for i in range(fpt*tpr*nrun):
    t = np.floor(i/fpt).astype(int)
    tmp[0][i], tmp[1][i] = pos[0][t], pos[1][t]

In [17]:
err = allXY - tmp
hit = np.zeros(fpt*tpr*nrun)
for i in range(fpt*tpr*nrun):
    hit[i] = abs(err[0][i]) <= bx*.5 and abs(err[1][i]) <= bx*.5
hit.shape

(87300,)

---

In [8]:
def calc_hit(file_behav, tpr, nrun):
    datum = scipy.io.loadmat(file_behav)
    
    ## frame per trial
    fpt = int(datum['nSampleTrial'][0][0])
    ## second per trial
    spt = int(fpt/60.)
    ## a number of trials per run
    # tpr = datum['nTrialperRun'][0][0]
#     tpr = 97  # 12 trial/block * 8 block/run + 1 = 97 trial/run
    ## target ID
    tmp = datum['targetID'][0]
    targetID = tmp[tmp!=0][:tpr*nrun]
#     nrun = 3

    bx = datum['boxSize'][0][0]
    pos = bx*GA.convert_ID(targetID)
    
    allXY = datum['allXY']
    
    xFrame, yFrame = np.array([datum['xFrame'].squeeze(), datum['yFrame'].squeeze()]).astype(int)
    
    ## target position
    tmp = np.zeros(allXY.shape)
    for i in range(fpt*tpr*nrun):
        t = np.floor(i/fpt).astype(int)
        tmp[0][i], tmp[1][i] = pos[0][t], pos[1][t]
        
    err = allXY - tmp
    hit = np.zeros(fpt*tpr*nrun)
    for i in range(fpt*tpr*nrun):
        hit[i] = abs(err[0][i]) <= bx*.5 and abs(err[1][i]) <= bx*.5
        
    return hit

---
Validation Check

In [6]:
dir_work = '/home/sungbeenpark/Github/labs/GA'
dir_data = join(dir_work, 'behav_data')

In [8]:
tmp = {}
list_behav = glob(join(dir_data, '*-fmri.mat'))
for fname in list_behav:
    subj = fname.split('/')[-1]
    hit = calc_hit(fname, tpr=97, nrun=3)
    tmp[subj] = hit.mean()
list_behav = glob(join(dir_data, '*-refmri.mat'))
for fname in list_behav:
    subj = fname.split('/')[-1]
    hit = calc_hit(fname, tpr=97, nrun=3)
    tmp[subj] = hit.mean()

In [9]:
tmp

{'GA07-fmri.mat': 0.27786941580756014,
 'GA33-fmri.mat': 0.09865979381443299,
 'GA01-fmri.mat': 0.3658762886597938,
 'GA29-fmri.mat': 0.25882016036655214,
 'GA44-fmri.mat': 0.23670103092783504,
 'GA35-fmri.mat': 0.2714318442153494,
 'GA28-fmri.mat': 0.38333333333333336,
 'GA30-fmri.mat': 0.5107903780068729,
 'GA05-fmri.mat': 0.3947079037800687,
 'GA14-fmri.mat': 0.16170675830469644,
 'GA08-fmri.mat': 0.4406872852233677,
 'GA11-fmri.mat': 0.18878579610538374,
 'GA26-fmri.mat': 0.2666323024054983,
 'GA13-fmri.mat': 0.36508591065292095,
 'GA02-fmri.mat': 0.44687285223367695,
 'GA20-fmri.mat': 0.2831615120274914,
 'GA23-fmri.mat': 0.4567812142038946,
 'GA15-fmri.mat': 0.3280526918671249,
 'GA18-fmri.mat': 0.46665521191294385,
 'GA19-fmri.mat': 0.28607101947308133,
 'GA38-fmri.mat': 0.2753150057273769,
 'GA12-fmri.mat': 0.26347079037800686,
 'GA21-fmri.mat': 0.1761282932416953,
 'GA36-fmri.mat': 0.3206300114547537,
 'GA27-fmri.mat': 0.4475715922107675,
 'GA42-fmri.mat': 0.20628865979381444,

In [11]:
res = []
for subj in GA.list_subj:
    res.append(tmp['GA'+subj+'-refmri.mat']-tmp['GA'+subj+'-fmri.mat'])
res

[0.4169530355097365,
 0.3595303550973655,
 0.34859106529209627,
 0.4112142038946162,
 0.2130698739977091,
 0.47028636884306985,
 0.34537227949599086,
 0.330446735395189,
 0.5466666666666666,
 0.41928980526918674,
 0.22038946162657508,
 0.31498281786941584,
 0.26674684994272624,
 0.5388545246277204,
 0.16390607101947308,
 0.5282016036655212,
 0.29249713631156926,
 0.296758304696449,
 0.47463917525773197,
 0.15111111111111108,
 0.22020618556701033,
 0.16646048109965633,
 0.6834020618556701,
 0.4012600229095074,
 0.37454753722794953,
 0.4102749140893471,
 0.5280870561282932,
 0.4738144329896907,
 0.412680412371134,
 0.4428178694158076]

In [13]:
tmp = GA.load_1D(join(dir_work, 'scripts', 'RewardRate_improvement.txt'))
tmp

array([0.417, 0.36 , 0.347, 0.413, 0.213, 0.475, 0.346, 0.329, 0.545,
       0.419, 0.219, 0.318, 0.265, 0.539, 0.164, 0.526, 0.292, 0.297,
       0.475, 0.151, 0.222, 0.166, 0.687, 0.402, 0.374, 0.412, 0.53 ,
       0.474, 0.414, 0.443])

In [16]:
res - tmp

array([-0.00047, -0.00055,  0.00192, -0.00151,  0.0002 , -0.00496,
       -0.00017,  0.00107,  0.00147, -0.00004,  0.00103, -0.00263,
        0.00198, -0.00016, -0.00018,  0.00215,  0.00026, -0.00038,
       -0.00034,  0.00049, -0.00155,  0.00075, -0.00368, -0.00095,
        0.00055, -0.0014 , -0.00181,  0.00021, -0.00085,  0.00031])

---

In [5]:
dir_work = '/home/sungbeenpark/Github/labs/GP'
dir_data = join(dir_work, 'behav_data')

In [21]:
# list_subj = []
# tmp = glob(join(dir_data, '*.mat'))
# for s in tmp:
#     list_subj.append(s.split('/')[-1].split('-')[-2])
# list_subj = sorted(list(set(list_subj)))
# list_subj

['GP01',
 'GP02',
 'GP03',
 'GP04',
 'GP05',
 'GP06',
 'GP07',
 'GP08',
 'GP09',
 'GP10',
 'GP11',
 'GP12',
 'GP13',
 'GP14',
 'GP15',
 'GP16',
 'GP17',
 'GP18',
 'GP19',
 'GP20',
 'GP21']

In [6]:
list_behav = glob(join(dir_data, '*-fmri.mat'))
list_behav

['/home/sungbeenpark/Github/labs/GP/behav_data/GP18-fmri.mat',
 '/home/sungbeenpark/Github/labs/GP/behav_data/GP17-fmri.mat',
 '/home/sungbeenpark/Github/labs/GP/behav_data/GP10-fmri.mat',
 '/home/sungbeenpark/Github/labs/GP/behav_data/GP09-fmri.mat',
 '/home/sungbeenpark/Github/labs/GP/behav_data/GP08-fmri.mat',
 '/home/sungbeenpark/Github/labs/GP/behav_data/GP20-fmri.mat',
 '/home/sungbeenpark/Github/labs/GP/behav_data/GP11-fmri.mat']

In [10]:
tmp = {}
for fname in list_behav:
    subj = fname.split('/')[-1]
    print(fname)
    hit = calc_hit(fname, tpr=97, nrun=3)
    for block in range(8*3):
        run = block // 8
        tmp[subj, block] = hit[ 300*(12*block + run) : 300*(12*(block+1) + run) ].mean()

/home/sungbeenpark/Github/labs/GP/behav_data/GP18-fmri.mat
/home/sungbeenpark/Github/labs/GP/behav_data/GP17-fmri.mat
/home/sungbeenpark/Github/labs/GP/behav_data/GP10-fmri.mat


MatReadError: Mat file appears to be empty

In [11]:
tmp

{('GP18-fmri.mat', 0): 0.0022222222222222222,
 ('GP18-fmri.mat', 1): 0.12111111111111111,
 ('GP18-fmri.mat', 2): 0.06472222222222222,
 ('GP18-fmri.mat', 3): 0.22972222222222222,
 ('GP18-fmri.mat', 4): 0.3363888888888889,
 ('GP18-fmri.mat', 5): 0.1963888888888889,
 ('GP18-fmri.mat', 6): 0.23833333333333334,
 ('GP18-fmri.mat', 7): 0.1802777777777778,
 ('GP18-fmri.mat', 8): 0.2822222222222222,
 ('GP18-fmri.mat', 9): 0.18944444444444444,
 ('GP18-fmri.mat', 10): 0.16055555555555556,
 ('GP18-fmri.mat', 11): 0.1663888888888889,
 ('GP18-fmri.mat', 12): 0.14944444444444444,
 ('GP18-fmri.mat', 13): 0.2608333333333333,
 ('GP18-fmri.mat', 14): 0.15666666666666668,
 ('GP18-fmri.mat', 15): 0.14555555555555555,
 ('GP18-fmri.mat', 16): 0.295,
 ('GP18-fmri.mat', 17): 0.21166666666666667,
 ('GP18-fmri.mat', 18): 0.17527777777777778,
 ('GP18-fmri.mat', 19): 0.155,
 ('GP18-fmri.mat', 20): 0.12166666666666667,
 ('GP18-fmri.mat', 21): 0.034444444444444444,
 ('GP18-fmri.mat', 22): 0.021666666666666667,
 ('GP